<h1>Предсказание вероятности дефолта по кредиту</h1>

In [252]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

In [253]:
# 1. Создаем искусственный датасет
np.random.seed(42)
data_size = 1000
data = pd.DataFrame({
    'credit_score': np.random.randint(300, 850, data_size),  # Кредитный рейтинг
    'income': np.random.randint(20000, 150000, data_size),   # Доход
    'num_loans': np.random.randint(1, 10, data_size),        # Количество кредитов
    'default': np.random.choice([0, 1], data_size, p=[0.8, 0.2])  # Метка дефолта
})

In [254]:
# 2. Разделение на обучающие и тестовые данные
X = data[['credit_score', 'income', 'num_loans']]
y = data['default']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Балансировка данных с помощью SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

In [255]:
# from sklearn.model_selection import RandomizedSearchCV
#
# scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
#
# # Определение пространства поиска гиперпараметров
# param_grid = {
#     'n_estimators': [100, 200, 300, 500],  # Количество деревьев
#     'max_depth': [3, 5, 7, 10],           # Максимальная глубина деревьев
#     'learning_rate': [0.01, 0.05, 0.1, 0.3],  # Скорость обучения
#     'subsample': [0.6, 0.7, 0.8],         # Доля выборки для каждого дерева
#     'colsample_bytree': [0.6, 0.7, 0.8],  # Доля признаков для каждого дерева
#     'min_child_weight': [1, 3, 5, 10],    # Минимальная сумма весов узла
#     'scale_pos_weight': [scale_pos_weight]  # Балансировка классов
# }
#
# # 3. Создание модели
# xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
#
# # RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     estimator=xgb,
#     param_distributions=param_grid,
#     scoring='f1',  # Оптимизируем F1-Score
#     n_iter=50,     # Количество итераций
#     cv=3,          # Количество фолдов для кросс-валидации
#     verbose=2,
#     random_state=42,
#     n_jobs=-1      # Используем все процессоры
# )
#
# # Запуск поиска
# random_search.fit(X_train, y_train)
#
# # Вывод лучших параметров
# print("Лучшие параметры:", random_search.best_params_)
#
# # Итоговая модель с оптимальными параметрами
# model = random_search.best_estimator_

In [256]:
# 3. Создание модели
model = XGBClassifier(scale_pos_weight=len(y_train[y_train == 0]) / len(y_train[y_train == 1]))
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [257]:
# 5. Прогнозируем на тестовых данных
y_pred_rf = model.predict(X_test)
y_pred_proba_rf = model.predict_proba(X_test)[:, 1]

In [258]:
# 6. Оцениваем производительность
accuracy_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(y_test, y_pred_proba_rf)

In [259]:
print(f"Точность (Accuracy): {accuracy_rf:.2f}")
print(f"F1-Score: {f1_rf:.2f}")
print(f"ROC-AUC: {roc_auc_rf:.2f}")
print("\nОтчет классификации:")
print(classification_report(y_test, y_pred_rf))

Точность (Accuracy): 0.70
F1-Score: 0.17
ROC-AUC: 0.52

Отчет классификации:
              precision    recall  f1-score   support

           0       0.81      0.83      0.82       244
           1       0.18      0.16      0.17        56

    accuracy                           0.70       300
   macro avg       0.49      0.49      0.49       300
weighted avg       0.69      0.70      0.70       300



In [260]:
new_client = pd.DataFrame({
    'credit_score': [350],  # Кредитный рейтинг клиента
    'income': [50000],      # Годовой доход клиента
    'num_loans': [30]        # Количество текущих кредитов
})

# Прогноз дефолта для нового клиента
default_prediction = model.predict(new_client)[0]  # 0 - нет дефолта, 1 - дефолт
default_probability = model.predict_proba(new_client)[0][1]  # Вероятность дефолта

# Вывод результатов
print(f"Результат: {'Дефолт' if default_prediction == 1 else 'Нет дефолта'}")
print(f"Вероятность дефолта: {default_probability:.2f}")

Результат: Нет дефолта
Вероятность дефолта: 0.02
